# Bereiten wir das Script vor

In [15]:
!pip install textract 
#Imports
import os #Files in listen abgreifen
import tqdm #anzeigen, wo wir stehen
import textract #das ist das instrument, um text aus pdfs raus zu ziehen
import pandas as pd


## Vorbereitung

In [13]:
#Als Vorbereitung, machen aus dem Inhalt des Folders eine Liste. Es handelt sich hierbei
#um eine Liste aller pdfs im Ordner pdfs. Es sind die Swissmedic Recalls (#ImplantFiles)
pdfs = os.listdir('pdfs')

In [14]:
pdfs

['Vk_20180226_08documents0.pdf',
 'Vk_20180226_08documents1.pdf',
 'Vk_20180226_08documents2.pdf',
 'Vk_20180326_03documents0.pdf',
 'Vk_20180413_17documents0.pdf',
 'Vk_20180413_17documents1.pdf',
 'Vk_20180413_17documents2.pdf',
 'Vk_20180413_17documents3.pdf',
 'Vk_20180417_10documents0.pdf',
 'Vk_20180417_10documents1.pdf',
 'Vk_20180509_08documents0.pdf',
 'Vk_20180509_08documents1.pdf',
 'Vk_20180509_08documents2.pdf',
 'Vk_20180514_49documents0.pdf',
 'Vk_20180514_49documents1.pdf',
 'Vk_20180514_49documents2.pdf',
 'Vk_20180516_01documents0.pdf',
 'Vk_20180516_01documents1.pdf',
 'Vk_20180516_07documents0.pdf',
 'Vk_20180516_07documents1.pdf',
 'Vk_20180516_07documents2.pdf',
 'Vk_20180516_12documents0.pdf',
 'Vk_20180516_14documents0.pdf',
 'Vk_20180516_14documents1.pdf',
 'Vk_20180517_02documents0.pdf',
 'Vk_20180517_02documents1.pdf',
 'Vk_20180517_02documents2.pdf',
 'Vk_20180517_02documents3.pdf',
 'Vk_20180517_06documents0.pdf',
 'Vk_20180517_12documents0.pdf',
 'Vk_20180

## Fortschritt anzeigen

In [10]:
from tqdm import tqdm
for i in tqdm(range(1000)): #100000000000
    elem = i

100%|██████████| 1000/1000 [00:00<00:00, 722657.48it/s]


## Ein File auslesen

In [16]:
text = textract.process("pdfs/"+pdfs[1]) #method='pdfminer', encoding='ISO-8859-1'

ShellError: The command `pdftotext pdfs/Vk_20180226_08documents1.pdf -` failed with exit code 127
------------- stdout -------------
------------- stderr -------------


In [17]:
str(text) #weil der text ein bytes ist, muss ichs in einen string verwandeln

NameError: name 'text' is not defined

## Alle Files auslesen

In [18]:
lst = []
mistakes = []
for pdf in tqdm(pdfs):
    try:
        text = textract.process("pdfs/"+pdf, method='pdfminer', encoding='ISO-8859-1') #damit ziehen wir den text raus. pdf ist der pfad, dann die methode, dann das endcoding. das alles brauchts. 
        lst.append(text) #dann den text in die liste packen
    except:
        mistakes.append(pdf) #wenn er keinen text hat, tut der das ins mistakes rein


 33%|███▎      | 27/82 [00:16<00:19,  2.83it/s]

Vk_20180517_12documents0.pdf



 48%|████▊     | 39/82 [00:25<00:40,  1.05it/s]

Vk_20180523_05documents2.pdf



 55%|█████▍    | 45/82 [00:28<00:16,  2.27it/s]

...


 72%|███████▏  | 59/82 [00:35<00:07,  3.06it/s]

Vk_20180530_11documents3.pdf
Vk_20180524_02documents0.pdf



 73%|███████▎  | 60/82 [00:35<00:06,  3.54it/s]

Vk_20180326_03documents0.pdf



100%|██████████| 82/82 [00:50<00:00,  2.73it/s]

## Bilder auslesen

In [21]:
text = textract.process("pdfs/"+pdfs[0], method='tesseract') #Nicht ideal, aber gut. tesseract nimmt bilder und wandelt das in text um

## Alles zusammen

In [ ]:
#Rohe Analyse
for pdf in tqdm(pdfs):
    try:
        text = textract.process("pdfs/"+pdf, method='pdfminer', encoding='ISO-8859-1') # ich encodiere das zeug zuerst
        text = text.decode("ISO-8859-1").replace("\n", " ") #ich ersetze leere zeilen mit leerem abstand
        if len(text) < 40:
            text = textract.process("pdfs/"+pdf, method='tesseract', encoding='ISO-8859-1') #alles, was kleiner ist als 40 zeichen, ist ein bild
            text = text.decode("ISO-8859-1").replace("\n", " ") #dann decodiere ich es wieder
            
        mini_dict = {'Text':text,
                     'File':pdf}
        lst.append(mini_dict)
    except:
        mini_dict = {'Text':'Fehlermeldung',
                     'File':pdf}
        lst.append(mini_dict)


 59%|█████▊    | 48/82 [00:37<00:40,  1.18s/it]

In [ ]:
df=pd.DataFrame(lst)

In [ ]:
def length(elem):
    elem = len(elem)
    reutrn elem
df['Text Länge'] = df['Text'].apply(length) #und das ist dann eine weitere kolonne, mit der ich arbeiten kann. 
#ich kann das dann so weiter auseinanderdröseln, bis ich erhalte, was ich benötige. 

In [ ]:
#Erste Säuberung
df = pd.DataFrame(lst)
def date(elem):
    elem = elem[3:11]
    return elem
df['date'] = df['File'].apply(date)
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df.index = df['date']
#Suchen wir noch nach Implantaten
df['implant'] = df['Text'].str.contains('implant')
df.to_csv('datafile.csv')